In [15]:
import matplotlib.pyplot as plt 
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.constraints import maxnorm 
from keras.optimizers import SGD 
from keras.utils import np_utils 
from keras import backend as K 

In [16]:
K.set_image_data_format('channels_last') #N x C x H x W

In [17]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255.0
x_test /= 255.0

print(y_train)

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

num_classes = y_test.shape[1]

[[6]
 [9]
 [9]
 ...
 [9]
 [1]
 [1]]


In [18]:
print(y_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [19]:
model = Sequential()

model.add(Conv2D(32, (3,3), input_shape=(32,32,3), activation='relu', padding='same'))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
print(model.summary())



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 conv2d_7 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 16, 16, 32)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 conv2d_9 (Conv2D)           (None, 16, 16, 64)        36928     
                                                      

In [20]:
lrate = 0.01
epochs = 40
decay = lrate/epochs
sgd = SGD(learning_rate=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics =['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=32)

Epoch 1/40
1563/1563 [==============================] - 23s 14ms/step - loss: 1.9795 - accuracy: 0.2645 - val_loss: 1.6128 - val_accuracy: 0.4063
Epoch 2/40
1563/1563 [==============================] - 12s 8ms/step - loss: 1.5697 - accuracy: 0.4224 - val_loss: 1.3816 - val_accuracy: 0.4977
Epoch 3/40
1563/1563 [==============================] - 11s 7ms/step - loss: 1.3533 - accuracy: 0.5105 - val_loss: 1.2080 - val_accuracy: 0.5669
Epoch 4/40
1563/1563 [==============================] - 12s 7ms/step - loss: 1.2023 - accuracy: 0.5685 - val_loss: 1.0175 - val_accuracy: 0.6339
Epoch 5/40
1563/1563 [==============================] - 11s 7ms/step - loss: 1.0787 - accuracy: 0.6205 - val_loss: 0.9420 - val_accuracy: 0.6661
Epoch 6/40
1563/1563 [==============================] - 12s 8ms/step - loss: 0.9757 - accuracy: 0.6547 - val_loss: 0.9137 - val_accuracy: 0.6770
Epoch 7/40
1563/1563 [==============================] - 11s 7ms/step - loss: 0.8997 - accuracy: 0.6858 - val_loss: 0.8195 - val_a

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print('Accuracy: %.2f%%' % (scores[1]*100))

In [ ]:
def summarize_diagnostics(history):
  fig, ax = plt.subplots(1,2)
  #plt.subplots(211)
  ax[0].set_title('Mean Square Error')
  ax[0].plot(history.history['loss'], color='blue', label='train')
  ax[0].plot(history.history['val_loss'], color='orange', label='test')

  #plt.subplots(212)
  ax[1].set_title('Classification Accuracy')
  ax[1].plot(history.history['accuracy'], color='blue', label='train')
  ax[1].plot(history.history['val_accuracy'], color='orange', label='test')

  # filename = sys.argv[0].split('/')[-1]
  # plt.savefig(filename + '_plot.png')
  plt.show()
  #plt.close()

summarize_diagnostics(history)

In [ ]:
print(history.history.keys())